## Python imports

In [ ]:
from jinja2 import Template
import os
import sys

### _Load Django into Jupyter_

In [ ]:
sys.path.append('/bookstore')
sys.path.append('/bookstore/project')
os.environ['DJANGO_SETTINGS_MODULE'] = 'project.settings'
os.environ['DJANGO_ALLOW_ASYNC_UNSAFE'] = 'true'
import django
django.setup()

## Adding LogLevels Objects

In [ ]:
with open("/templates/dbfill_bookstore.py", "r+") as file:
    data = file.read()
    if not os.path.exists("/bookstore/bookstore/scripts"):
        os.makedirs("/bookstore/bookstore/scripts")
    with open("/bookstore/bookstore/scripts/dbfill_bookstore.py", "w") as file:
        write_file = file.write(data)

print(open("/bookstore/bookstore/scripts/dbfill_bookstore.py").read())

In [ ]:
%%bash

cd /bookstore
python manage.py runscript dbfill_bookstore

### _let's check . . ._

In [ ]:
from bookstore.models import *

for b in Book.objects.all():
    print("%s, written by %s" % (b,b.author))

## Configuring Serialization

In [ ]:
file = '/bookstore/bookstore/serializers.py'
    
with open(file, 'w') as file:
    template = Template(open('/templates/serializers.j2', 'r').read())
    print(template.render())
    file.write(template.render())

## Challenge #3
### Explain how serialization helps with data collection

In [ ]:
book = Book.objects.last()
book.title = f"{book.title} - Second Edition"
book.changeReason = "New edition"
book.changed_by_id = 2
book.__dict__
book.save()

In [ ]:
from rest_framework import serializers
from rest_framework.reverse import reverse
from django.contrib.auth.models import User
from bookstore.models import *
import json


# class HistoricalRecordField(serializers.ListField):
#     child = serializers.DictField()

#     def to_representation(self, data):
#         return super().to_representation(data.values())

    
class BookSpecialDetailSerializer(serializers.ModelSerializer):
    author = serializers.SerializerMethodField()
    changed_by = serializers.SerializerMethodField()
    history = serializers.SerializerMethodField()

    class Meta:
        model = Book
        fields = ('id', 'title', 'author', 'changed_by', 'history')
#         fields = '__all__'
        read_only_fields = ['id']
        depth = 1

    def get_author(self, obj):
        obj.author.changed_by_username = obj.author.changed_by.username
        return obj.author.__dict__
    
    def get_changed_by(self, obj):
        return obj.changed_by.username

    def get_history(self, obj):
        history = []
        for h in obj.history.all():
            h.changed_by_username = h.changed_by.username
            history.append(h.__dict__)
        return history


print(f"Looking at {book.title}, written by {book.author}\n\n")

serialized_book = BookSpecialDetailSerializer(book).data
print(json.dumps(serialized_book, indent=4, sort_keys=True, default=str))